# Poisson Equation

**Goal:** give details for solving the Poisson equation numerically for an arbitray potential. 

The Poisson equation reads
$$
\nabla^2 \phi(x) = 4\pi \rho(x)
$$

## Fast-Fourier Transform

For periodic boundary condition it is conveniant to solve the Poisson equation using Fourier transform.

In [12]:
using FFTW
using GLMakie

function poisson_solver_1d(f, Lx; G=1)
    Nx = length(f)
    dx = Lx / Nx

    # Compute wave numbers
    kx = 2π / Lx * [0:Nx/2; -Nx/2+1:-1]

    # Compute Fourier coefficients of f
    fh = fft(f)

    # Compute Fourier coefficients of the solution
    uh = zeros(ComplexF64, Nx)
    for i in 1:Nx
        if i != 1
            uh[i] = -4π * G * fh[i] / kx[i]^2
        end
    end

    # Compute the solution in real space
    u = real(ifft(uh))

    return u
end

Lx = 100
f = [exp(-1e-2 * (x - 50)^2) for x in 1:Lx]

100-element Vector{Float64}:
 3.737571327944256e-11
 9.859505575991516e-11
 2.549381880391969e-10
 6.461431773106108e-10
 1.6052280551856116e-9
 3.908938434264864e-9
 9.330287574504972e-9
 2.182957795125478e-8
 5.006218020767049e-8
 1.1253517471925912e-7
 ⋮
 2.182957795125478e-8
 9.330287574504972e-9
 3.908938434264864e-9
 1.6052280551856116e-9
 6.461431773106108e-10
 2.549381880391969e-10
 9.859505575991516e-11
 3.737571327944256e-11
 1.3887943864964021e-11

In [13]:
ϕ = poisson_solver_1d(f, Lx; G = 1e-3)

fig = lines(ϕ)
lines!(f)
fig

In [24]:
function poisson_solver_2d(f, Lx, Ly; G=1e-3)
    Nx, Ny = size(f)
    dx = Lx / Nx
    dy = Ly / Ny

    # Compute wave numbers
    kx = 2π / Lx * [0:Nx/2; -Nx/2+1:-1]
    ky = 2π / Ly * [0:Ny/2; -Ny/2+1:-1]

    # Compute Fourier coefficients of f
    fh = fft(f)

    # Compute Fourier coefficients of the solution
    uh = zeros(ComplexF64, Nx, Ny)
    for i in 1:Nx
        for j in 1:Ny
            if (i, j) != (1, 1)
                uh[i, j] = fh[i, j] / (kx[i]^2 + ky[j]^2)
            end
        end
    end

    # Compute the solution in real space
    u = real(ifft(uh))

    return u
end

Lx = 100
Ly = 100
f = [exp(-1e-2 * ((x - 50)^2 + (y - 50)^2)) for x in 1:Lx, y in 1:Ly]

heatmap(f)

In [25]:
ϕ = poisson_solver(f, Lx, Ly)

heatmap(ϕ)

size(ϕ) = (100, 100)
